In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import matplotlib.pyplot as plt
from tabulate import tabulate
import seaborn as sns
from collections import Counter
import plotly.graph_objects as go
import plotly.express as px
import calmap
import os
import csv

In [ ]:
def read_files_by_country(dir_name, country_code):
    try:
        json_file = os.path.join(dir_name, f"{country_code}_category_id.json")
        csv_file = os.path.join(dir_name, f"{country_code}_youtube_trending_data.csv")

        with open(json_file, 'r', encoding='utf-8') as json_data:
            json_content = json.load(json_data)
            # print(f"Contents of {json_file}:")
            # print(json_content)

        # for japanese
        # videos = pd.read_csv(csv_file, encoding="latin")
        videos = pd.read_csv(csv_file)
        # print(videos)

    except FileNotFoundError:
        print(f"Files for {country_code} not found.")

    return json_content, videos

country_code = 'RU'
json_content, videos = read_files_by_country('/Users/<USERID>/Desktop/Data/archive2023', country_code)

In [ ]:
videos['description'] = videos['description'].apply(lambda x: '' if pd.isnull(x)  else x)

##converting dates and times and creating a new column for publish time

videos['publishedAt'] = pd.to_datetime(videos['publishedAt'])
videos['trending_date'] = pd.to_datetime(videos['trending_date'])
videos.dropna(subset=['publishedAt'], inplace=True)
videos.dropna(subset=['trending_date'], inplace=True)
# videos.head()

videos['publish_date'] = videos['publishedAt'].dt.date
videos['publish_time'] = videos['publishedAt'].dt.time
videos['trending_date'] = videos['trending_date'].dt.date


In [ ]:


##creating new column which has the time until the video begins to trend

json_df = pd.DataFrame(json_content['items'])
# Convert 'id' column to int64
json_df['id'] = json_df['id'].astype('int64')


# Extract 'title' from 'snippet' and assign it a name
json_df['category_title'] = json_df['snippet'].apply(lambda x: x['title'])

# Merge the DataFrames with the 'title' column
videos = videos.merge(json_df[['id', 'category_title']], left_on='categoryId', right_on='id')

# Drop the 'id' column from the merged DataFrame
videos = videos.drop(columns=['id'])

trending_repeat = videos.groupby(['video_id'])['category_title'].value_counts().reset_index(name='Trending Days')
# trending_repeat.head()

repeat_all = trending_repeat.pivot_table(index='Trending Days', columns='category_title', values='video_id', aggfunc='count').fillna(0)
# repeat_all

repeat_all_percentage = repeat_all.divide(repeat_all.sum(axis=1), axis=0) * 100
# repeat_all_percentage.head()

In [ ]:
sort_trending = videos.groupby('video_id').size().reset_index(name='days_trending')
# drop duplicates
video_noduplicates = videos.drop_duplicates(subset='video_id', keep='first')
merge_videos = pd.merge(video_noduplicates, sort_trending, on='video_id')